In [1]:
import pandas as pd
import pickle
import shapely

# redistrict drop

In [34]:
#to_correct = 'Datasets/data_FEC_NATIONALPOLL_2004_2018.p'
to_correct = 'Datasets/data_FEC_NATIONALPOLL_DEMOGRAPHICSIMPUTED_2004_2018.p'

overlap = pickle.load(open('Datasets/all_overlap_data.p', 'rb'))
dataset = pickle.load(open(to_correct, 'rb'))
dataset.shape

(3431, 42)

In [35]:
dataset[dataset['year'] == 2012].shape

(423, 42)

In [36]:
#drop redistricted districts
new_dataset_name = to_correct[:-2] + '_REDISTRICTDROP.p'
border_change = overlap['border_change']
border_change = border_change[~border_change.isnull()]
border_change = border_change[border_change == 1] #1: border changed, 0: border not changed
for cand_index in dataset.index.values:
    if cand_index in border_change.index:
        dataset = dataset[dataset.index != cand_index]
dataset.shape

(3007, 42)

In [37]:
dataset[dataset['year'] == 2012].shape

(99, 42)

In [38]:
centroids = overlap['centroid']
for cand_index in dataset.index.values:
    lon, lat = centroids[centroids.index == cand_index].values[0][0]
    dataset.loc[dataset.index == cand_index, 'longitude'] = lon
    dataset.loc[dataset.index == cand_index, 'latitude'] = lat
dataset.shape, dataset.columns

((3007, 44),
 Index(['district', 'state', 'year', 'party', 'candidatevotes', 'totalvotes',
        'candidate', 'national_poll', 'national_poll_prev',
        'national_poll_delta_subtract', 'national_poll_delta_divide',
        'previous_party', 'dem_win_margin_prev', 'rep_win_margin_prev',
        'margin_signed_minus_prev', 'margin_signed_divide_prev',
        'margin_unsigned_minus_prev', 'margin_unsigned_divide_prev',
        'dem_win_prev', 'rep_win_prev', 'dem_win', 'rep_win', 'female_pct',
        'age18_24_pct', 'age25_34_pct', 'median_age', 'unmarried_partner_pct',
        'male_living_alone_pct', 'bachelors_deg_or_higher_pct',
        'past_year_births_to_unmarried_women_pct', 'civilian_veteran_pct',
        'live_same_house_past_year_pct',
        'foreign_born_proportion_from_LatinAmerica',
        'speak_other_language_at_home_pct', 'labor_force_unemployed_pct',
        'public_transit_commuter_pct', 'no_health_insurance_pct',
        'poverty_rate_pct', 'median_housing_v

In [39]:
dataset[dataset['year']!=2018].isnull().values.any()

False

In [40]:
pickle.dump(dataset, open(new_dataset_name, 'wb'))

# demographics mix

In [146]:
to_correct = 'Datasets/data_FEC_NATIONALPOLL_DEMOGRAPHICSIMPUTED_2004_2018.p'

overlap = pickle.load(open('district_shapefiles/all_overlap_data.p', 'rb'))
dataset = pickle.load(open(to_correct, 'rb'))
dataset.shape

(3431, 42)

In [147]:
pop_map = overlap['population_overlap'] #2014 -> 2012
inv_pop_map = overlap['inverse_population_overlap'] #2014 -> 2016

In [148]:
d_cols = ['female_pct', 'age18_24_pct', 'age25_34_pct', 'median_age',
       'unmarried_partner_pct', 'male_living_alone_pct',
       'bachelors_deg_or_higher_pct',
       'past_year_births_to_unmarried_women_pct', 'civilian_veteran_pct',
       'live_same_house_past_year_pct',
       'foreign_born_proportion_from_LatinAmerica',
       'speak_other_language_at_home_pct', 'labor_force_unemployed_pct',
       'public_transit_commuter_pct', 'no_health_insurance_pct',
       'poverty_rate_pct', 'median_housing_value', 'median_household_income',
       'food_stamp_benefits_pct', 'foreign_to_native_born_ratio']

In [149]:
set(dataset['year'].values), dataset.shape

({2004.0, 2006.0, 2008.0, 2010.0, 2012.0, 2014.0, 2016.0, 2018.0}, (3431, 42))

In [150]:
dataset.head()

,district,state,year,party,candidatevotes,totalvotes,candidate,national_poll,national_poll_prev,national_poll_delta_subtract,...,foreign_born_proportion_from_LatinAmerica,speak_other_language_at_home_pct,labor_force_unemployed_pct,public_transit_commuter_pct,no_health_insurance_pct,poverty_rate_pct,median_housing_value,median_household_income,food_stamp_benefits_pct,foreign_to_native_born_ratio
AK_01_2004,1.0,AK,2004.0,republican,213216,299996,Don Young,3.680556,-0.989011,4.669567,...,19.2,16.5,6.6,1.2,19.9,9.9,241400,64576.0,10.6,0.074201
AK_01_2006,1.0,AK,2006.0,republican,132743,234645,Don Young,10.736842,3.680556,7.056287,...,19.2,16.5,6.6,1.2,19.9,9.9,241400,64576.0,10.6,0.074201
AK_01_2008,1.0,AK,2008.0,republican,158939,316978,Don Young,9.824324,10.736842,-0.912518,...,19.2,16.5,6.6,1.2,19.9,9.9,241400,64576.0,10.6,0.074201
AK_01_2010,1.0,AK,2010.0,republican,175384,254335,Don Young,-2.622642,9.824324,-12.446966,...,19.2,16.5,6.6,1.2,19.9,9.9,241400,64576.0,10.6,0.074201
AK_01_2012,1.0,AK,2012.0,republican,185296,289804,Don Young,1.988304,-2.622642,4.610946,...,17.0,16.6,5.3,1.6,20.5,10.1,245100,67712.0,11.4,0.077029


In [151]:
dataset = dataset.sort_values('year', ascending=False)
dataset.head()

,district,state,year,party,candidatevotes,totalvotes,candidate,national_poll,national_poll_prev,national_poll_delta_subtract,...,foreign_born_proportion_from_LatinAmerica,speak_other_language_at_home_pct,labor_force_unemployed_pct,public_transit_commuter_pct,no_health_insurance_pct,poverty_rate_pct,median_housing_value,median_household_income,food_stamp_benefits_pct,foreign_to_native_born_ratio
MN_04_2018,4.0,MN,2018.0,democrat,None,None,None,7.168919,3.175439,3.99348,...,16.8,20.5,2.8,6.0,4.7,11.7,248000,68684.0,10.2,0.173159
LA_06_2018,6.0,LA,2018.0,republican,None,None,None,7.168919,3.175439,3.99348,...,47.0,7.6,3.4,0.5,6.8,12.3,189000,62374.0,12.9,0.047121
KY_03_2018,3.0,KY,2018.0,democrat,None,None,None,7.168919,3.175439,3.99348,...,34.8,10.0,3.8,3.0,5.3,14.0,169700,53677.0,12.5,0.089853
KY_04_2018,4.0,KY,2018.0,republican,None,None,None,7.168919,3.175439,3.99348,...,36.2,4.9,2.9,0.8,4.6,11.9,172500,62095.0,10.6,0.034157
KY_05_2018,5.0,KY,2018.0,republican,None,None,None,7.168919,3.175439,3.99348,...,22.1,2.0,3.7,0.3,6.0,29.1,81500,31731.0,24.7,0.006692


In [152]:
dataset.dtypes

district                                     float64
state                                         object
year                                         float64
party                                         object
candidatevotes                                object
totalvotes                                    object
candidate                                     object
national_poll                                float64
national_poll_prev                           float64
national_poll_delta_subtract                 float64
national_poll_delta_divide                   float64
previous_party                                object
dem_win_margin_prev                          float64
rep_win_margin_prev                          float64
margin_signed_minus_prev                     float64
margin_signed_divide_prev                    float64
margin_unsigned_minus_prev                   float64
margin_unsigned_divide_prev                  float64
dem_win_prev                                 f

In [153]:
#fix the demographics
dataset_dcorr = dataset.copy()
prev_year = 2020
for index, row in dataset_dcorr.iterrows():
    if row['year'] < prev_year:
        prev_year = row['year']
        print('on {}.'.format(prev_year))
    if row['year'] < 2010:
        for d_col in d_cols:
            cval = 0
            for cindex, cratio in inv_pop_map[index].items():
                cval += dataset_dcorr.loc[cindex, d_col] * cratio
            dataset_dcorr.loc[index, d_col] = cval

on 2018.0.
on 2016.0.
on 2014.0.
on 2012.0.
on 2010.0.
on 2008.0.
on 2006.0.
on 2004.0.


In [154]:
test = 'GA_09_2004'
dataset_dcorr[dataset_dcorr.index == test]

,district,state,year,party,candidatevotes,totalvotes,candidate,national_poll,national_poll_prev,national_poll_delta_subtract,...,foreign_born_proportion_from_LatinAmerica,speak_other_language_at_home_pct,labor_force_unemployed_pct,public_transit_commuter_pct,no_health_insurance_pct,poverty_rate_pct,median_housing_value,median_household_income,food_stamp_benefits_pct,foreign_to_native_born_ratio
GA_09_2004,9.0,GA,2004.0,republican,197869,266331,Charlie Norwood,3.680556,-0.989011,4.669567,...,52.4752,12.6949,6.6495,0.9342,18.7778,17.9226,159650,47032.777,11.8435,0.105604


In [155]:
dataset[dataset.index == test]

,district,state,year,party,candidatevotes,totalvotes,candidate,national_poll,national_poll_prev,national_poll_delta_subtract,...,foreign_born_proportion_from_LatinAmerica,speak_other_language_at_home_pct,labor_force_unemployed_pct,public_transit_commuter_pct,no_health_insurance_pct,poverty_rate_pct,median_housing_value,median_household_income,food_stamp_benefits_pct,foreign_to_native_born_ratio
GA_09_2004,9.0,GA,2004.0,republican,197869,266331,Charlie Norwood,3.680556,-0.989011,4.669567,...,72.4,14.7,7.3,0.3,18.6,15.9,163100,45925.0,11.5,0.107734


In [159]:
pickle.dump(dataset_dcorr, open('Datasets/data_FEC_NATIONALPOLL_DEMOGRAPHICSIMPUTED_DISTRICTMIXED_2004_2018.p', 'wb'))
dataset_dcorr.to_csv('Datasets/data_FEC_NATIONALPOLL_DEMOGRAPHICSIMPUTED_DISTRICTMIXED_2004_2018.csv')

In [156]:
set(dataset_dcorr['year'].values), dataset_dcorr.shape

({2004.0, 2006.0, 2008.0, 2010.0, 2012.0, 2014.0, 2016.0, 2018.0}, (3431, 42))

In [157]:
# m_cols = ['dem_win_margin_prev', 'rep_win_margin_prev',
#         'margin_signed_minus_prev', 'margin_signed_divide_prev',
#         'margin_unsigned_minus_prev', 'margin_unsigned_divide_prev']

In [158]:
# #fix margins
# dataset_acorr = dataset_dcorr.copy()
# prev_year = 2020
# for index, row in dataset_dcorr.iterrows():
    

SyntaxError: unexpected EOF while parsing (<ipython-input-158-530ac488a403>, line 5)